# UB NN Training
## Loading Dataset

In [1]:
csv_path = '../input/life-expectancy-who-pre-process/Life_expectancy_last_column.csv'

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
df = pd.read_csv(csv_path, index_col=0)
df.head()

,Year,Adult Mortality,Infant Deaths,Alcohol,Expenditure(%),Hepatitis B,Measles,BMI,Under-five Deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,Thinness 1-19 Years,Thinness 5-9 Years,ICOR,Schooling,Life Expectancy
0,2015,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,2014,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,2013,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,2012,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,2011,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


## Loading Similarity Information

In [2]:
import pickle as pkl

fileName =  '../input/a1-le-data-to-similar-events/Similarity'
fileObject2 = open(fileName, 'rb')
Imported_data = pkl.load(fileObject2)
fileObject2.close()

#print('Similarity Thresholds:',Imported_data[:,0])

#print('Most Similarity at:',Imported_data[:,1])

Similar_event_count = len(Imported_data[0,:])-1
print('Number of Similar Samples Considered', Similar_event_count)

Similarity_thresholds = Imported_data[:,0]

Sample_density = Similar_event_count/ Similarity_thresholds/100
#Sample_density = number of samples with less than 1% (of range) deviation

Similar_samples = Imported_data[:,1:Similar_event_count+1]

print('Target Shape', Sample_density.shape, 'Similar Samples Matrix Shape:', Similar_samples.shape)

input_=[]
output_=[]
for row in df.iloc:
        input_.append((row[0:len(row)-1]).astype(float))
        output_.append(row[-1])

Number of Similar Samples Considered 100
Target Shape (1649,) Similar Samples Matrix Shape: (1649, 100)


## Load UB Correction Model

In [3]:
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data

PATH = '../input/a4-bound-correction-nn-training/best_model.pt'

num_input = 1
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x
net_load = Net(n_feature=num_input, n_hidden=400, n_output=1)     # define the network
#net_load = net_load.to(device)

net_load.load_state_dict(torch.load(PATH))



<All keys matched successfully>

In [4]:
Uncertainty_Bound = torch.tensor(0.4) # The expected UB

In [5]:

f = open('../input/a4-bound-correction-nn-training/min_x.pckl', 'rb')
min_UBx = pkl.load(f)
f.close()

f = open('../input/a4-bound-correction-nn-training/max_x.pckl', 'rb')
max_UBx = pkl.load(f)
f.close()

Uncertainty_Bound = (Uncertainty_Bound-min_UBx)/(max_UBx-min_UBx) # Normalizing

new_shape = (Uncertainty_Bound.shape[0], 1)
Uncertainty_Bound = Uncertainty_Bound.view(new_shape)

Uncertainty_Bound_corrected = net_load(Uncertainty_Bound)

f = open('../input/a4-bound-correction-nn-training/min_y.pckl', 'rb')
min_UBy = pkl.load(f)
f.close()

f = open('../input/a4-bound-correction-nn-training/max_y.pckl', 'rb')
max_UBy = pkl.load(f)
f.close()

Uncertainty_Bound_corrected = Uncertainty_Bound_corrected*(max_UBy-min_UBy)+min_UBy # DeNormalizing

print(Uncertainty_Bound_corrected)


tensor([[0.3888]], grad_fn=<AddBackward0>)


## Applying Corrected UB to Traditional System

In [6]:
higher_count=0;
bound_values = []
for iter1 in range(len(output_)):
    indexes=Similar_samples[iter1];
    Probability_density =[]
    for iter2 in indexes:
        Probability_density.append(output_[iter2.astype(np.int64)])
    Probability_density = np.sort(Probability_density)
    prediction_bound = Probability_density[np.round(Similar_event_count*Uncertainty_Bound_corrected.detach().numpy()).astype(np.int64)-1]
    bound_values.append(prediction_bound)
    if prediction_bound > output_[iter1]:
        higher_count = higher_count + 1;
        
print('Statistical UB location:', higher_count/len(output_))

Statistical UB location: 0.3953911461491813


## Data Preparation for NN Training

In [7]:
x=[]
y=[]
i_val=[]
o_val=[]
for iter1 in range(len(output_)):
    if iter1%9>0:
        x.append(input_[iter1])
        y.append(bound_values[iter1])
        continue
    i_val.append(input_[iter1])
    o_val.append(bound_values[iter1])

x = torch.tensor(x).float()  
y = torch.tensor(y).float() 

new_shape = (len(y), 1)
y = y.view(new_shape)
print('Shapes:', x.shape, y.shape)

i_val = torch.tensor(i_val).float()  
o_val = torch.tensor(o_val).float()   
new_shape = (len(o_val), 1)
o_val = o_val.view(new_shape)


max_y = torch.max(y[:,0])
min_y =torch.min(y[:,0])

max_x = torch.max(x,dim=0)
min_x = torch.min(x,dim=0)

print(max_y, min_y, max_x.values, min_x.values)

range_y = max_y - min_y
range_x = max_x.values - min_x.values

print(range_x, range_y)

    #Normalizing
x = (x - min_x.values)/range_x
y = (y - min_y)/range_y

    #Normalizing
i_val = (i_val - min_x.values)/range_x
o_val = (o_val - min_y)/range_y

Shapes: torch.Size([1465, 19]) torch.Size([1465, 1])
tensor(81.2000) tensor(50.) tensor([2.0150e+03, 7.2300e+02, 1.6000e+03, 1.7870e+01, 1.8961e+04, 9.9000e+01,
        1.3144e+05, 7.7100e+01, 2.1000e+03, 9.9000e+01, 1.4390e+01, 9.9000e+01,
        5.0600e+01, 1.1917e+05, 1.2939e+09, 2.7200e+01, 2.8200e+01, 9.3600e-01,
        2.0700e+01]) tensor([2.0000e+03, 1.0000e+00, 0.0000e+00, 1.0000e-02, 0.0000e+00, 2.0000e+00,
        0.0000e+00, 2.0000e+00, 0.0000e+00, 3.0000e+00, 7.4000e-01, 2.0000e+00,
        1.0000e-01, 1.6813e+00, 3.4000e+01, 1.0000e-01, 1.0000e-01, 0.0000e+00,
        4.2000e+00])
tensor([1.5000e+01, 7.2200e+02, 1.6000e+03, 1.7860e+01, 1.8961e+04, 9.7000e+01,
        1.3144e+05, 7.5100e+01, 2.1000e+03, 9.6000e+01, 1.3650e+01, 9.7000e+01,
        5.0500e+01, 1.1917e+05, 1.2939e+09, 2.7100e+01, 2.8100e+01, 9.3600e-01,
        1.6500e+01]) tensor(31.2000)


##  NN Training for Direct Input to UB

In [8]:
# torch can only train on Variable, so convert them to Variable
x, y = Variable(x), Variable(y)
num_input=19
 
class Net2(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net2, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.hidden2 = torch.nn.Linear(n_hidden, n_hidden)   # hidden layer
        self.predict = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = F.relu(self.hidden2(x))
        x = self.predict(x)             # linear output
        return x


    
net = Net2(n_feature=num_input, n_hidden=500, n_output=1)     # define the network
print(net)  # net architecture
optimizer = torch.optim.Adam(net.parameters(), lr=0.1)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss


minimum_train_loss = 1e5
minimum_val_loss = 1e5
EPOCH = 1000

# start training
for epoch in range(EPOCH):
  
    prediction = net(x)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    if loss<minimum_train_loss:
        minimum_train_loss =loss
        net_opt = net
    
    if epoch%200 == 199:
        prediction = net_opt(i_val)
        loss_val = loss_func(prediction, o_val)
        if loss_val<minimum_val_loss:
            minimum_val_loss = loss_val
            net_opt_val = net_opt
        if loss_val>minimum_val_loss:
            net_opt = net_opt_val
        print ("Epoch [{}/{}], Loss: {:.4f}, Minimum Loss {:.6f}, Val Loss {:.6f}  "  .format(epoch+1, EPOCH, loss, minimum_train_loss, minimum_val_loss))

Net2(
  (hidden): Linear(in_features=19, out_features=500, bias=True)
  (hidden2): Linear(in_features=500, out_features=500, bias=True)
  (predict): Linear(in_features=500, out_features=1, bias=True)
)
Epoch [200/1000], Loss: 0.0481, Minimum Loss 0.048116, Val Loss 0.049064  
Epoch [400/1000], Loss: 0.0481, Minimum Loss 0.048116, Val Loss 0.049064  
Epoch [600/1000], Loss: 0.0481, Minimum Loss 0.048116, Val Loss 0.049064  
Epoch [800/1000], Loss: 0.0481, Minimum Loss 0.048116, Val Loss 0.049064  
Epoch [1000/1000], Loss: 0.0481, Minimum Loss 0.048116, Val Loss 0.049064  


## Statistical performance of NN on Training Set

In [9]:
higher_count=0;
x_0=[]
prediction_=[]

for iter1 in range(len(output_)):
    x_in = input_[iter1]
    x_0.append(x_in[1])
    Net_input = torch.tensor(input_[iter1]).float() 
    Net_input = (Net_input - min_x.values)/range_x # Normalizing
    
    prediction_bound = net_opt_val(Net_input) # Apply Net
    
    prediction_bound = prediction_bound*range_y+min_y # Denormalizing    
       
    prediction_.append(float(prediction_bound))
    
    if prediction_bound > output_[iter1]:
        higher_count = higher_count + 1;
        
print('Statistical UB location for Direct NN:', higher_count/len(output_))

Statistical UB location for Direct NN: 0.40873256519102485
